## ResNext 29 4x64

 - Training Dataset:  RandAugment, N=3, M=20
 - Sagemaker Notebook must be of type, conda_pytorch_p36
 
#### Install Requirements

In [8]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

#### Without Containers

In [4]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

  Cloning https://github.com/NVIDIA/apex.git to /tmp/pip-install-gskewhf0/apex
  Running command git clone -q https://github.com/NVIDIA/apex.git /tmp/pip-install-gskewhf0/apex
  Running command git submodule update --init --recursive -q
  Created wheel for fvcore: filename=fvcore-0.1.1.post20200623-py3-none-any.whl size=41179 sha256=0d0aaa4ace3de166ef11214ea46eba085436aa50062d7349eb2e0afa46c9854b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/27/fc/86/b29aea030f5468db673ec86033a9579cc50e02979aa0c78ebe
  Created wheel for apex: filename=apex-0.1-py3-none-any.whl size=192130 sha256=95369c882569b08fb0d3c8610e76f3c7508b1952ca61548c7f6599a23b28f474
  Stored in directory: /tmp/pip-ephem-wheel-cache-__xt74d1/wheels/2a/45/23/6b4f2d6323a65ee0022d22a96d7bf580138e689f17cc48235c
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=2521e69be0aee9dee08f0c7dd6b7cff7e41cae1b0f8372ce92f2bb624efb3c7a
  Stored in directory: /home/ec2-user/.cache/pip/wheels/93

In [5]:
# Need to add this to requirements.txt
!pip install tensorboard

     |████████████████████████████████| 3.0 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 2.6 MB/s  eta 0:00:01
     |████████████████████████████████| 104 kB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 1.8 MB/s  eta 0:00:01
     |████████████████████████████████| 3.0 MB 25.3 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 66.6 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 54.5 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 55.5 MB/s eta 0:00:01
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121931 sha256=676973d2208585efd2d403e4ae9b81cdfdb00be33eba02ac87572bfbd3cddd06
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/af/84/3962a6af7b4ab336e951b7877dcfb758cf94548bb1771e0679
Successfully built absl-py
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Train the model per the settings specified for ResNext 29_4x64d in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/resnext.yaml \
    model.resnext.cardinality 4 \
    train.batch_size 128 \
    dataset.name CIFAR10_RA_3_20 \
    train.base_lr 0.1 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00 \
    scheduler.epochs 400

# Number of epochs should be 300!

[2020-06-28 15:43:18] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_3_20
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck

In [7]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
#!python train.py --config configs/cifar/resnet.yaml \
!python train.py --config /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/config.yaml \
    model.resnext.cardinality 4 \
    test.batch_size 128 \
    train.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr .001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50 \
    scheduler.epochs 50

#### Set LEARNING RATE based on ending LR


[2020-07-01 00:19:54] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: resnext
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_t

In [9]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - without training on unaugmented data
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/test_results_0400_cifar10

[2020-07-02 17:28:06] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:35<00:00,  2.24it/s]
[2020-07-02 17:28:43] __main__ INFO: Elapsed 35.30
[2020-07-02 17:28:43] __main__ INFO: Loss 1.3829 Accuracy 0.6392


In [10]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset - using model refined on 50 epochs
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50/test_results_0050_cifar10

[2020-07-02 17:31:51] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|███████████████████████████████████████████| 79/79 [00:35<00:00,  2.23it/s]
[2020-07-02 17:32:28] __main__ INFO: Elapsed 35.37
[2020-07-02 17:32:28] __main__ INFO: Loss 0.3423 Accuracy 0.9018


In [11]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - without training on unaugmented data
# write the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/resnext.yaml \
    model.resnext.cardinality 4 \
    test.batch_size 128 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/test_results_0400_CIFAR101

[2020-07-02 17:33:37] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.17it/s]
[2020-07-02 17:33:45] __main__ INFO: Elapsed 7.37
[2020-07-02 17:33:45] __main__ INFO: Loss 1.9102 Accuracy 0.4935


In [12]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset - using model refined on 50 epochs
# write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/resnext.yaml \
   model.resnext.cardinality 4 \
   test.batch_size 128 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50/test_results_0050_cifar101

[2020-07-02 17:34:34] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 16/16 [00:07<00:00,  2.19it/s]
[2020-07-02 17:34:42] __main__ INFO: Elapsed 7.32
[2020-07-02 17:34:42] __main__ INFO: Loss 0.6259 Accuracy 0.8185


In [13]:
import pandas as pd
a = pd.Series(['resnext_29_4x64d_ra_3_20', 400, 'cifar10', 1.3829, 0.7241]) #Loss 1.3829 Accuracy 0.6392
b = pd.Series(['resnext_29_4x64d_ra_3_20', 400, 'cifar10.1', 1.9102, 0.4935]) #Loss 1.9102 Accuracy 0.4935

c = pd.Series(['resnext_29_4x64d_ra_3_20_refined400', 50, 'cifar10', 0.3423, 0.9018]) #Loss 0.3423 Accuracy 0.9018
d = pd.Series(['resnext_29_4x64d_ra_3_20_refined400', 50, 'cifar10.1', 0.6259, 0.8185]) #Loss 0.6259 Accuracy 0.8185

df_results = pd.concat([a,b,c,d], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy_300epochs'] = df_results.apply((lambda row: 96.4 if row[2] == 'cifar10' else 89.6), axis=1)
df_results['Original_CI_300epochs'] = df_results.apply((lambda row: (96.0, 96.7) if row[2] == 'cifar10' else (88.2, 90.9)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/resnext_29_4x64d_ra_3_20/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy_300epochs,Original_CI_300epochs
0,resnext_29_4x64d_ra_3_20,400,cifar10,1.3829,0.7241,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d_ra_3_20,400,cifar10.1,1.9102,0.4935,89.6,"(88.2, 90.9)"
2,resnext_29_4x64d_ra_3_20_refined400,50,cifar10,0.3423,0.9018,96.4,"(96.0, 96.7)"
3,resnext_29_4x64d_ra_3_20_refined400,50,cifar10.1,0.6259,0.8185,89.6,"(88.2, 90.9)"


In [3]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['resnext_29_4x64d', 'resnext_29_4x64d', 'resnext_29_4x64d', 'resnext_29_4x64d'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.6746, 0.2311, 0.1517, 0.3742],
           'Accuracy': [0.8019, 0.9321, 0.9535, 0.8905],
           'Original_Accuracy': [96.4, 96.4, 96.4, 89.6],
           'Original_CI': [(96.0, 96.7), (96.0, 96.7), (96.0, 96.7), (88.2, 90.9)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/resnext_29_4x64d/exp00/results.csv')
df.head()

,Model,Testset,Epoch,Loss,Accuracy,Original_Accuracy,Original_CI
0,resnext_29_4x64d,cifar10,100,0.6746,0.8019,96.4,"(96.0, 96.7)"
1,resnext_29_4x64d,cifar10,200,0.2311,0.9321,96.4,"(96.0, 96.7)"
2,resnext_29_4x64d,cifar10,300,0.1517,0.9535,96.4,"(96.0, 96.7)"
3,resnext_29_4x64d,cifar10.1,300,0.3742,0.8905,89.6,"(88.2, 90.9)"


In [60]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/resnext_29_4x64d/exp00/test_results_0300/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[-1.2041907 , -1.8999833 , -0.24285015, ..., -1.5008752 ,
        -1.8426697 , -2.8560946 ],
       [ 0.5460079 ,  2.220384  , -1.9393705 , ..., -2.6070693 ,
        11.327686  , -1.2085156 ],
       [-1.3446747 ,  2.1730833 , -1.1615647 , ..., -2.2299995 ,
        10.984515  , -0.75660706],
       ...,
       [-2.4790986 , -1.3337001 ,  0.61669415, ..., -0.83421385,
        -1.8529658 , -1.7280097 ],
       [-0.90489024,  9.350766  ,  1.0618937 , ..., -2.3210623 ,
        -0.9061641 , -1.8115013 ],
       [-1.4560711 , -1.0518838 , -1.4613396 , ..., 12.668192  ,
        -2.1191459 , -0.8881919 ]], dtype=float32)

In [54]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-may29'
prefix = 'sagemaker/results/original-models/'
path = '/home/ec2-user/SageMaker/experiments/'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)